## Data <a name="data"></a>

Based on definition of our problem, factors that will influence my decission are:
* number of existing cafes in the neighborhood (any type of cafe)
* number of and distance to cafes in the neighborhood, if any
* distance of neighborhood from city center

I decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of cafes and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Kiev center will be obtained using geocoder **Nominatum** of well known Kiev location (Khreshchatyk Street)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Kiev city center.

Let's first find the latitude & longitude of Kiev city center, using specific, well known address and geocoder Nominatum.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes ssss
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [2]:
address = 'Khreshchatyk, Kiev, Ukraine'

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
kiev_center = [latitude, longitude]
print('Coordinate of {}: {}'.format(address, kiev_center))

Coordinate of Khreshchatyk, Kiev, Ukraine: [50.4478094, 30.5287584]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Khreshchatyk. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
!pip install shapely
!pip install pyproj

Requirement not upgraded as not directly required: shapely in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: pyproj in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [4]:
import shapely.geometry

import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=36, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=36, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Kiev center longitude={}, latitude={}'.format(kiev_center[1], kiev_center[0]))
x, y = lonlat_to_xy(kiev_center[1], kiev_center[0])
print('Kiev center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Kiev center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Kiev center longitude=30.5287584, latitude=50.4478094
Kiev center UTM X=324554.0920229856, Y=5591340.281641376
Kiev center longitude=30.5287584, latitude=50.4478094


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [5]:
kiev_center_x, kiev_center_y = lonlat_to_xy(kiev_center[1], kiev_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = kiev_center_x - 6000
x_step = 600
y_min = kiev_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(kiev_center_x, kiev_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [6]:
!pip install folium

import folium

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly require

In [7]:
map_kiev = folium.Map(location=kiev_center, zoom_start=13)
folium.Marker(kiev_center, popup='Khreshchatyk').add_to(map_kiev)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_kiev) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_kiev)
    #folium.Marker([lat, lon]).add_to(map_kiev)
map_kiev

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Khreshchatyk. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [8]:
addr = location.address
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(kiev_center[0], kiev_center[1], addr))

Reverse geocoding check
-----------------------
Address of [50.4478094, 30.5287584] is: Хрещатик, 6, Героїв Небесної Сотні алея, Черепанова гора, Печерськ, Печерський район, Київ, 1001, Україна


In [10]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = geolocator.geocode([lat, lon]).address
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Україна', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [11]:
addresses[150:170]

['10, Русанівська набережна, Русанівка, Дніпровський район, Київ, 02147',
 'Школа №136, Ентузіастів вулиця, Русанівка, Дніпровський район, Київ, 02147',
 'Русанівка, Дніпровський район, Київ, 020154',
 '32а, Залізнична вулиця, Чоколівка, Солом’янський район, Київ, 03056',
 'Церква Сергія Радонезького, 14, Уманська вулиця, Чоколівка, Солом’янський район, Київ, 03087',
 '19, Юліуса Фучика вулиця, Солом’янка, Солом’янський район, Київ, 03043',
 'Бюро перепусток, Повітрофлотський проспект, Солом’янка, Солом’янський район, Київ, 03049',
 'Київ-Пасажирський, Східний підземний перехід, Кучмин Яр, Солом’янський район, Київ, 03031',
 'Тіп-топ сервіс, 27, Симона Петлюри вулиця, Кудрявець, Київ, Шевченківський район, Київ, 01000-06999',
 'ТТС - 1, 10/18, Льва Толстого вулиця, Го́род Володимира, Київ, Шевченківський район, Київ, 01000-06999',
 'Володимирська вулиця, Го́род Володимира, Київ, Шевченківський район, Київ, 01046',
 'Хрещатик вулиця, Го́род Володимира, Київ, Шевченківський район, Київ, 

Looking good. Let's now place all this into a Pandas dataframe.

In [12]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Distance from center,Latitude,Longitude,X,Y
0,"3, Васильківська вулиця, Голосіїв, Голосіївськ...",5992.495307,50.395913,30.506130,322754.092023,5.585625e+06
1,"4а, Владислава Заремби провулок, Добрий Шлях, ...",5840.376700,50.396094,30.514563,323354.092023,5.585625e+06
2,"2а, Листопадна вулиця, Добрий Шлях, Голосіївсь...",5747.173218,50.396274,30.522997,323954.092023,5.585625e+06
3,"Продукти, 29, Науки проспект, Добрий Шлях, Гол...",5715.767665,50.396453,30.531431,324554.092023,5.585625e+06
4,"42, Столєтова вулиця, Багринова гора, Голосіїв...",5747.173218,50.396632,30.539865,325154.092023,5.585625e+06
5,"Потерна №7, Багринова гора, Голосіївський райо...",5840.376700,50.396810,30.548299,325754.092023,5.585625e+06
6,"Столичне шосе, Китаїв, Голосіївський район, Ки...",5992.495307,50.396988,30.556733,326354.092023,5.585625e+06
7,"33а, Краматорська вулиця, Ширма, Голосіївський...",5855.766389,50.400310,30.493233,321854.092023,5.586144e+06
8,"20, Криворізька вулиця, Ширма, Голосіївський р...",5604.462508,50.400491,30.501668,322454.092023,5.586144e+06
9,"Дитсадок № 41 Веселка, Деміївська вулиця, Дем...",5408.326913,50.400672,30.510102,323054.092023,5.586144e+06


...and let's now save/persist this data into local file.

In [13]:
df_locations.to_pickle('./locations.pkl')

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on cafes in each neighborhood.

We're interested in venues in 'food' category, but only those that are places for coffee - coffee shops, cafes, bakeries etc. So we will include in out list only venues that have 'cafe' in category name, and we'll make sure to detect and include all the subcategories of specific 'coffee' category, as we need info on places for coffee in the neighborhood.

Foursquare credentials are defined in the hidden cell bellow.

In [15]:
# Category IDs corresponding to coffee places were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

coffee_cafe_categories = ['4bf58dd8d48988d1e0931735','4bf58dd8d48988d16a941735', '4bf58dd8d48988d1dc931735', '4bf58dd8d48988d179941735', '4bf58dd8d48988d143941735',
                          '4bf58dd8d48988d16d941735', '4bf58dd8d48988d1d0941735'
                  ]

def is_cafe(categories, specific_filter=None):
    cafe_words = ['coffee', 'breakfast', 'lunch', 'bakery', 'tea']
    cafe = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in cafe_words:
            if r in category_name:
                cafe = True
        if 'fast food' in category_name:
            cafe = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            cafe = True
    return cafe, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [16]:
# Let's now go over our neighborhood locations and get nearby cafes; we'll also maintain a dictionary of all found cafes and all found coffee places

import pickle

def get_cafes(lats, lons):
    cafes = {}
    cofee_cafes = {}
    location_cafes = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any cafe (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        area_cafes = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_caf, is_coffee = is_cafe(venue_categories, specific_filter=coffee_cafe_categories)
            if is_caf:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                cafe = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_coffee, x, y)
                if venue_distance<=300:
                    area_cafes.append(cafe)
                cafes[venue_id] = cafe
                if is_coffee:
                    coffee_cafes[venue_id] = cafe
        location_cafes.append(area_cafes)
        print(' .', end='')
    print(' done.')
    return cafes, coffee_cafes, location_cafes

# Try to load from local file system in case we did this before
cafes = {}
coffee_cafes = {}
location_cafes = []
loaded = False
try:
    with open('cafes_350.pkl', 'rb') as f:
        cafes = pickle.load(f)
    with open('coffee_cafes_350.pkl', 'rb') as f:
        coffee_cafes = pickle.load(f)
    with open('location_cafes_350.pkl', 'rb') as f:
        location_cafes = pickle.load(f)
    print('Cafe data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    cafes, coffee_cafes, location_cafes = get_cafes(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('cafes_350.pkl', 'wb') as f:
        pickle.dump(cafes, f)
    with open('coffee_cafes_350.pkl', 'wb') as f:
        pickle.dump(coffee_cafes, f)
    with open('location_cafes_350.pkl', 'wb') as f:
        pickle.dump(location_cafes, f)
        

Cafe data loaded.


In [17]:
import numpy as np

print('Total number of cafes:', len(cafes))
print('Total number of Coffee cafes:', len(coffee_cafes))
print('Percentage of Coffee cafes: {:.2f}%'.format(len(coffee_cafes) / len(cafes) * 100))
print('Average number of cafes in neighborhood:', np.array([len(r) for r in location_cafes]).mean())

Total number of cafes: 475
Total number of Coffee cafes: 463
Percentage of Coffee cafes: 97.47%
Average number of cafes in neighborhood: 1.13186813187


In [18]:
print('List of all cafes')
print('-----------------------')
for r in list(cafes.values())[:10]:
    print(r)
print('...')
print('Total:', len(cafes))

List of all cafes
-----------------------
('5a1c3e3aa795bd0b4f99bf20', 'Coffee Is', 50.454105, 30.505863, 'Київ, Україна', 302, True, 322952.33245810214, 5592094.519484019)
('4ec65ae30aaf9c3ccaf01385', '"St. Georgia" Грузинский Лаваш', 50.449612126476254, 30.494666397392344, 'Vorovskogo 36 (Turgenivska), Київ, Україна', 238, True, 322140.77899719775, 5591621.785172384)
('5073ef88e4b0ed9e499d5295', 'Сонях', 50.488665748399804, 30.474978077520863, 'Московский пр., 9, Київ, Україна', 63, True, 320890.90358810616, 5596010.835355894)
('500ceb25e4b084af1ee5dc64', 'Форнетти', 50.44018757673831, 30.50865327632549, 'Україна', 243, True, 323098.4717046725, 5590540.624254762)
('557035b5498e61b3f1db9be8', 'Чудо Пекар', 50.42472924311726, 30.458338213956512, 'просп. Повітрофлотський, Київ, 03151, Україна', 319, True, 319467.4114821417, 5588943.125410499)
('5791c416498e9ef3065f7f66', 'Happy Cake', 50.43231600202966, 30.516329056718508, 'м. Олимпийская, Київ, Україна', 292, True, 323614.1740931149, 5

In [19]:
print('List of Coffee cafes')
print('---------------------------')
for r in list(coffee_cafes.values())[:10]:
    print(r)
print('...')
print('Total:', len(coffee_cafes.values()))

List of Coffee cafes
---------------------------
('5a1c3e3aa795bd0b4f99bf20', 'Coffee Is', 50.454105, 30.505863, 'Київ, Україна', 302, True, 322952.33245810214, 5592094.519484019)
('4ec65ae30aaf9c3ccaf01385', '"St. Georgia" Грузинский Лаваш', 50.449612126476254, 30.494666397392344, 'Vorovskogo 36 (Turgenivska), Київ, Україна', 238, True, 322140.77899719775, 5591621.785172384)
('5073ef88e4b0ed9e499d5295', 'Сонях', 50.488665748399804, 30.474978077520863, 'Московский пр., 9, Київ, Україна', 63, True, 320890.90358810616, 5596010.835355894)
('500ceb25e4b084af1ee5dc64', 'Форнетти', 50.44018757673831, 30.50865327632549, 'Україна', 243, True, 323098.4717046725, 5590540.624254762)
('557035b5498e61b3f1db9be8', 'Чудо Пекар', 50.42472924311726, 30.458338213956512, 'просп. Повітрофлотський, Київ, 03151, Україна', 319, True, 319467.4114821417, 5588943.125410499)
('5791c416498e9ef3065f7f66', 'Happy Cake', 50.43231600202966, 30.516329056718508, 'м. Олимпийская, Київ, Україна', 292, True, 323614.174093

In [20]:
print('Cafes around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_cafes[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Cafes around location {}: {}'.format(i+1, names))

Cafes around location
---------------------------
Cafes around location 101: 
Cafes around location 102: Pies & Friends Bakery
Cafes around location 103: 
Cafes around location 104: Хлібний, Palais Royal Café
Cafes around location 105: Кафе в Будинку футболу
Cafes around location 106: 
Cafes around location 107: Львівські круасани (Львівські Круасани), Волконский Хлебная Лавка
Cafes around location 108: Кафе "Аркада"/"Arkada" Cafe
Cafes around location 109: Лаврська Харчевня
Cafes around location 110: DeJavu


Let's now see all the collected cafes in our area of interest on map, and let's also show coffee places in different color.

In [21]:
map_Kiev = folium.Map(location=kiev_center, zoom_start=13)
folium.Marker(kiev_center, popup='Khreshchatyk').add_to(map_kiev)
for caf in cafes.values():
    lat = caf[2]; lon = caf[3]
    is_coffee = caf[6]
    color = 'red' if is_coffee else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_kiev)
map_kiev

Looking good. So now we have all the cafes in area within few kilometers from Khreshchatyk, and we know which ones are coffee places! We also know which cafes exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new coffee place!